In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
import pickle
import pyarrow

In [3]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

### data summary
- There are 68211 total records, after filtering by duration > 1 & <=60 there will be 65924 records, which is 96% of the data

In [4]:
def read_and_preprocess(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    
    categorical = ['PU_DO']#'DOLocationID',]
    #categorical = ['PULocationID','DOLocationID']#'DOLocationID',]
    numerical = ['trip_distance']
    
    df[categorical]= df[categorical].astype(str)
    df = df[(df.duration > 1) & (df.duration <=60)]
    return df

In [5]:
def read_and_preprocess_green_taxi(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    
    categorical = ['PU_DO']#'DOLocationID',]
    #categorical = ['PULocationID','DOLocationID']#'DOLocationID',]
    numerical = ['trip_distance']
    
    df[categorical]= df[categorical].astype(str)
    df = df[(df.duration > 1) & (df.duration <=60)]
    return df


df_train = read_and_preprocess_green_taxi('../data/green_tripdata_2023-01.parquet')
df_valid = read_and_preprocess_green_taxi('../data/green_tripdata_2023-02.parquet')

In [6]:
# df_train = read_and_preprocess('../data/yellow_tripdata_2023-01.parquet')
# df_valid = read_and_preprocess('../data/yellow_tripdata_2023-02.parquet')

In [7]:
len(df_train),len(df_valid)

(65924, 62547)

In [30]:
df_train['PU_DO'].dtype

dtype('O')

### Training pipeline

In [23]:
dv = DictVectorizer()

categorical = ['PULocationID','DOLocationID','PU_DO']
numerical = ['trip_distance']

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_valid[categorical + numerical].to_dict(orient='records')
x_val = dv.transform(val_dict)

In [24]:

target = 'duration'
y_train = df_train[target].values
y_valid = df_valid[target].values

In [47]:

#training
Linear_R = LinearRegression()
Linear_R.fit(X_train,y_train)
#prediction
y_pred = Linear_R.predict(x_val)
#evaluation
rmse = root_mean_squared_error(y_valid,y_pred)
print(rmse)

6.030871201958465


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


##### dump model to pickle

model_choice = 'linear_reg_yellow'
out_put_file = f'model_type_{model_choice}.bin'


with open(f'../models/{out_put_file}', 'wb') as f_out:
    pickle.dump((dv, Linear_R), f_out)

with open(f'../models/{out_put_file}', 'rb') as f_in:
    dv, Linear_R = pickle.load(f_in)

In [11]:
dv, Linear_R

(DictVectorizer(), LinearRegression())

In [42]:
inference_input = {
    "PULocationID": 140,
    "DOLocationID": 50,
    "PU_DO": '140_50',
    "trip_distance": 5.4
}

In [43]:
X_inf = dv.transform([inference_input])
y_pred = Linear_R.predict(X_inf)
print(y_pred)

[23.88872229]


## with Lasso Reg

In [17]:
#With Lasso
lr = Lasso(alpha=0.01)
lr.fit(X_train,y_train)
#prediction
y_pred = lr.predict(x_val)
#evaluation
rmse = root_mean_squared_error(y_valid,y_pred)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(7.7361261061027395)

In [18]:
#With Ridge
LR = Ridge()
LR.fit(X_train,y_train)
#prediction
y_pred = LR.predict(x_val)
#evaluation
rmse = root_mean_squared_error(y_valid,y_pred)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(7.348087107758644)